In [275]:
import random
import numpy as np
import torch
import torch.nn as nn
import importlib
import life.lib
import life.lib as lib
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)

2.0.0+cu117


In [276]:
network = lib.nets.PYRAMID()
network = lib.nets.create_livenet_pyramid()
train_x, train_y = lib.datasets.get_pyramid()
criterion = lib.nets.criterion

def create_optimizer(net: nn.Module):
    if net.__class__.__name__ == "LiveNet":
        print("LiveNet")
        net: lib.livenet.LiveNet
        optimizer = lib.livenet.LiveNetOptimizer(net, decay=0.0001)
    else:
        print("Torch")
        optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
    return optimizer

optimizer = create_optimizer(network)


LiveNet


In [293]:
params = [p for p in network.named_parameters()]
logits = network.forward(train_x)
for name, p in params:
    print(f"{name} {p.detach().numpy():.3f}")
pred = network.forward(train_x)
loss = criterion(pred, train_y)
print(loss)
pred.detach().numpy()
# torch.onnx.export(network, train_x, "/home/spometun/model.onnx", verbose=False)




n0 -2.370
n1 1.237
s0 -1.236
s1 -1.236
s2 -1.237
s3 3.711
n2 -0.831
s4 0.831
s5 0.831
s6 0.831
s7 2.488
tensor(0.1489, grad_fn=<DivBackward0>)


array([[ 2.218],
       [ 2.218],
       [-2.37 ],
       [-2.37 ],
       [-2.369],
       [-2.369],
       [ 1.765]], dtype=float32)

In [279]:
network.context.decay = 0.0005

In [292]:

n_epochs = 8000

for i in range(n_epochs):
    pred = network.forward(train_x)
    loss = criterion(pred, train_y)
    if i % 100 == 0:
        print(f"{loss.detach().numpy():.3f}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149
0.149


In [26]:
# params = [p for p in network.named_parameters()]
pred = network.forward(train_x)
loss = criterion(pred, train_y)
pred.detach().numpy()


array([[-0.696],
       [-0.705],
       [-0.692],
       [-0.705],
       [-0.708]], dtype=float32)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(all_losses)
plt.ylabel('Loss')
plt.show()

In [24]:
# list(network.parameters())[0].data = torch.Tensor([[10.], [-10]])
# list(network.parameters())[1].data = torch.Tensor([-5., 15])
np.set_printoptions(precision=2)
params = list(network.parameters())
par = []
for p in params:
    par.append(p.detach())
    print (par[-1])

tensor([-1.5000])
tensor([[15.2505],
        [15.9158]])
tensor([1.4539, 0.7973])


In [26]:
o1 = train_x @ par[1].T + par[2]
o1

tensor([[ 1.4539,  0.7973],
        [16.7044, 16.7131],
        [31.9548, 32.6288]])

In [27]:
a1 = torch.sigmoid(o1)
a1

tensor([[0.8106, 0.6894],
        [1.0000, 1.0000],
        [1.0000, 1.0000]])

In [56]:
o2 = a1[:, 0] + a1[:, 1] - 1
mseloss(o2, train_y)

tensor(0.4356)

In [49]:
o2 @ par[2].T

tensor([[3.7133],
        [3.7137],
        [3.7140],
        [3.5771]])

In [23]:
network.forward(train_x)

tensor([[0.0000],
        [0.5000],
        [0.5000]], grad_fn=<AddBackward0>)

In [9]:
1345639 * 5145

6923312655

In [48]:
import pickle
import numpy as np

class A:
    def __init__(self):
        self.v = 1
    def __del__(self):
        print(f"del {self.v}")

a = A()
b = A()
b.v = 2
l = [b]
print("start")
del a
print("after del")
l[0].v
a.v


del 2
del 1
start
del 1
after del


NameError: name 'a' is not defined